<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/Query_Decomposition/%5Bfinal_form%5Dsubquery_by_conjunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai ragas datasets faiss-cpu groq langchain_community evaluate anls sentence_transformers
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2

In [ ]:
from google.colab import drive
from langchain.vectorstores import FAISS
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from google.colab import drive, userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY2')

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data= [
    {
        "question": "Which American singer and songwriter has a mezzo-sprano vocal range, Tim Arm or Tori Amos?",
        "answer": "Sam Bankman-Fried",
        "decomposed": [
            "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges?",
            "Who is accused by prosecutors of committing fraud for personal gain in the cryptocurrency industry?",
            "What industry is the individual associated with who is facing a criminal trial on fraud and conspiracy charges?",
            "Who has been reported by The Verge and TechCrunch in relation to a criminal trial on fraud and conspiracy charges?"
        ],
        "supporting_facts_sentences": [
            "Before his fall, Bankman-Fried made himself out to be the Good Boy of crypto — the trustworthy face of a sometimes-shady industry.",
            "The highly anticipated criminal trial for Sam Bankman-Fried, former CEO of bankrupt crypto exchange FTX, started Tuesday to determine whether he’s guilty of seven counts of fraud and conspiracy.",
            "The prosecution painted Bankman-Fried as someone who knowingly committed fraud to achieve great wealth, power and influence, while the defense countered that the FTX founder acted in good faith, never meant to commit fraud or steal and basically got in over his head."
        ]
    }
]
# input 파일 형태 예시

In [ ]:
import re
import json
import openai

# Load JSON data from file
input_path = "/content/drive/MyDrive/Final/Dataset/multihop/without_binary/multihop_test_noBinary_v2.json"  #input 파일 경로
output_path = "/content/drive/MyDrive/Final/Dataset/multihop/without_binary/multihop_langchainv2_noBinary_v2.json"  # 결과 저장 경로
model = "gpt-4o"  # Replace with your model if needed

# 접속사와 콤마를 기준으로 분리하는 함수
def split_sentence(text):
    # 예외 처리할 접속사 패턴
    exception_patterns = [
#        r'\b(?:both|either|neither|not only)\b.*?\b(?:and|or|nor)\b',
        r'\b(?:not only)\b.*?\b(?:but also)\b'
    ]
    # 예외 패턴에 해당하는 부분을 찾아서 미리 제외
    for pattern in exception_patterns:
        text = re.sub(pattern, lambda match: match.group(0).replace(" ", "_"), text)  # 예외 접속사 사이의 공백을 "_"로 치환

    # 일반적인 접속사와 콤마를 기준으로 분리
    conjunctions = r'\b(?:and|or|but|so|nor|yet)\b'
    parts = re.split(fr'\s*(?:{conjunctions}|,)\s*', text)

    # "_"를 원래 공백으로 복구
    parts = [part.replace("_", " ").strip() for part in parts if part.strip()]
    return parts

# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.

Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "What is the name of the organization discussed in TechCrunch articles that, despite its financial instability, is recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations, and is planning to enhance its capabilities with the release of GPT-4 and associated APIs?"
->
- subquery: What is the name of the organization recognized for creating ChatGPT?
- subquery: What is the name of the organization recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations?
- subquery: What is the name of the organization planning to enhance ChatGPT with the release of GPT-4 and associated APIs?
- subquery: What is the name of the organization discussed in TechCrunch articles, despite its financial instability?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft

Another Example:
Original Query: "Which company, as reported by both TechCrunch and The Verge, has spent billions to maintain its default search engine status on various platforms and is also accused of harming news publishers’ revenue through its business practices?"
- subquery: Which company has spent billions to maintain its default search engine status on various platforms?
- subquery: Which company is accused of harming news publishers’ revenue through its business practices?
- subquery: Which company is reported by both TechCrunch and The Verge for these actions?

One more example:
Original Query: "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
->
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud?
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial conspiracy charges?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by The Verge?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by TechCrunch?
- subquery: Who is the individual is accused by prosecutors of committing fraud for personal gain?

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt
# LLM 호출 예시
def get_subqueries2(original_query, model="gpt-4o"):
    split_results = split_sentence(original_query)
    prompt = generate_subqueries2(original_query, split_results)

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # 각 서브쿼리의 앞에 있는 "subquery:"와 "-"를 제거하고 리스트로 반환
    subqueries = [
        query.replace("- subquery: ", "").strip()  # "subquery:" 제거
        for query in response.choices[0].message.content.splitlines() if query.strip()
    ]
    return subqueries

# JSON 데이터 처리 함수
def process_json(data):
    processed_entries = []
    tmp = 0
    for entry in data:
        tmp += 1
        if tmp < 100:
            continue

        processed_entry = {
            "question": entry["question"],
            "ground_truth": entry["answer"],
            "ground_evidence": entry.get("supporting_facts_sentences", []),
            "decomposed_queries": get_subqueries2(entry["question"])
        }
        processed_entries.append(processed_entry)
    return processed_entries

# Load and process JSON
with open(input_path, 'r') as file:
    data = json.load(file)

processed_data = process_json(data)

# Save the updated JSON file
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

print("JSON file updated and saved successfully.")


JSON file updated and saved successfully.


## 프롬프트 HotpotVer & MulithopRAG ver
hotpot과 Multihoop 각각 프롬프트를 다르게 줘서 해당하는 것 진행할때 아래에서 가져다 쓰면 됩니다.

In [ ]:
#######################MultihopRAG#####################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.

Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "What is the name of the organization discussed in TechCrunch articles that, despite its financial instability, is recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations, and is planning to enhance its capabilities with the release of GPT-4 and associated APIs?"
->
- subquery: What is the name of the organization recognized for creating ChatGPT?
- subquery: What is the name of the organization recognized for creating ChatGPT, which is both a priority and a platform for ongoing innovations?
- subquery: What is the name of the organization planning to enhance ChatGPT with the release of GPT-4 and associated APIs?
- subquery: What is the name of the organization discussed in TechCrunch articles, despite its financial instability?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft

Another Example:
Original Query: "Which company, as reported by both TechCrunch and The Verge, has spent billions to maintain its default search engine status on various platforms and is also accused of harming news publishers’ revenue through its business practices?"
- subquery: Which company has spent billions to maintain its default search engine status on various platforms?
- subquery: Which company is accused of harming news publishers’ revenue through its business practices?
- subquery: Which company is reported by both TechCrunch and The Verge for these actions?

One more example:
Original Query: "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
->
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud?
- subquery: Who is the individual associated with the cryptocurrency industry facing a criminal trial conspiracy charges?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by The Verge?
- subquery: Who is the individual associated with the cryptocurrency industry as reported by TechCrunch?
- subquery: Who is the individual is accused by prosecutors of committing fraud for personal gain?

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt

In [ ]:
#################hotpotQA############################
# 서브쿼리 생성 프롬프트
def generate_subqueries2(original_query, split_results):
    prompt = f"""
Given a query with multiple entities or phrases separated by conjunctions or commas, create each subquery by keeping as much of the original phrasing as necessary to maintain the intent and meaning of the original query.

Instructions:
1. For each phrase or section after a conjunction or comma, create a complete subquery that retains necessary context from the original query.
2. Ensure each subquery stands alone with full context so no information or intent is lost, and no subquery is overly simplified.
3. Maintain original terms where possible and ensure each subquery is grammatically complete.
When you fill in a sentence (subquery) in Separate Parts, always find it in the words in the original question and fill it in.


Original Query: "{original_query}"
Separated Parts:
{split_results}

Example:
Original Query: "Who has won more awards, Dan Schneider or Helen Hunt?"
->
- subquery: Who has won more awards than Helen Hunt?
- subquery: Who has won more awards than Dan Schneider?

Example:
Original Query: "profit of Apple, Tesla and Microsoft?"
->
- subquery: profit of Apple
- subquery: profit of Tesla
- subquery: profit of Microsoft

One more example:
Original Query: "Which American singer and songwriter has a mezzo-soprano vocal range, Tim Armstrong or Tori Amos?"
->
- subquery: Which American singer has a mezzo-soprano vocal range?
- subquery: Which American songwriter has a mezzo-soprano vocal range?
- subquery: Is Tim Armstrong an American singer and songwriter with a mezzo-soprano vocal range?
- subquery: Is Tori Amos an American singer and songwriter with a mezzo-soprano vocal range?

Now, based on the above examples, provide a list of subqueries for each part of the question while preserving the essential meaning.
- subquery: <- Don't say anything other than the format that starts with this form. And the decomposed query that is generated can't just generate the original question. Don't forget the purpose of dividing in the original question.
"""
    return prompt